In [9]:
# init
from pyquda.utils import gauge_utils
from pyquda.field import LatticeFermion
from pyquda import core, pyqcu, mpi
import os
import sys
from time import perf_counter
import cupy as cp
test_dir = os.path.dirname(os.path.abspath("./"))
sys.path.insert(0, os.path.join(test_dir, ".."))
os.environ["QUDA_RESOURCE_PATH"] = ".cache"
latt_size = [32, 32, 32, 64]
grid_size = [1, 1, 1, 1]
Lx, Ly, Lz, Lt = latt_size
Nd, Ns, Nc = 4, 4, 3
Gx, Gy, Gz, Gt = grid_size
latt_size = [Lx//Gx, Ly//Gy, Lz//Gz, Lt//Gt]
Lx, Ly, Lz, Lt = latt_size
Vol = Lx * Ly * Lz * Lt
mpi.init(grid_size)
device_latt_tmp0 = LatticeFermion(latt_size, cp.zeros(
    (Lt, Lz, Ly, Lx, Ns, Nc), cp.complex128))
device_latt_tmp1 = LatticeFermion(latt_size, cp.zeros(
    (Lt, Lz, Ly, Lx, Ns, Nc), cp.complex128))
latt_shape = (Lt, Lz, Ly, Lx//2, Ns, Nc)
param = pyqcu.QcuParam()
param.lattice_size = latt_size
mpi_dslash = core.getDslash(latt_size, -3.5, 0, 0, anti_periodic_t=False)
kappa = 0.125
U = gauge_utils.gaussGauge(latt_size, 0)
mpi_dslash.loadGauge(U)

Creating Gaussian distrbuted Lie group field with sigma = 1.000000e-01


In [10]:
# give ans first
ans_e = cp.random.random(latt_shape) + 1j * \
    cp.random.random(latt_shape)  # ans_e
ans_o = cp.random.random(latt_shape) + 1j * \
    cp.random.random(latt_shape)  # ans_o

In [11]:
# give x_o, b__o, r, r_tilde, p, v, s, t
x_o = cp.random.random(latt_shape) + 1j * cp.random.random(latt_shape)  # x_o
b_e = cp.zeros(latt_shape, cp.complex128)
b_o = cp.zeros(latt_shape, cp.complex128)
b__o = cp.zeros(latt_shape, cp.complex128)
r = cp.zeros(latt_shape, cp.complex128)
r_tilde = cp.zeros(latt_shape, cp.complex128)
p = cp.zeros(latt_shape, cp.complex128)
s = cp.zeros(latt_shape, cp.complex128)
v = cp.zeros(latt_shape, cp.complex128)
t = cp.zeros(latt_shape, cp.complex128)
# give r_norm2, MAX_ITER, TOL, rho_prev, rho, alpha, omega, beta, kappa
r_norm2 = 0
MAX_ITER = 1e3
TOL = 1e-9
rho_prev = 1
rho = 0
alpha = 1
omega = 1
beta = 0
kappa = 0.125

In [12]:
# define dslash
def dslash_eo(src_o):
    device_latt_tmp1.data[1, :] = src_o
    pyqcu.dslashQcu(device_latt_tmp0.even_ptr, device_latt_tmp1.odd_ptr,
                    U.data_ptr, param, 0)  # D_eo
    return device_latt_tmp0.data[0, :]


def dslash_oe(src_e):
    device_latt_tmp1.data[0, :] = src_e
    pyqcu.dslashQcu(device_latt_tmp0.odd_ptr, device_latt_tmp1.even_ptr,
                    U.data_ptr, param, 1)  # D_oe
    return device_latt_tmp0.data[1, :]


def dslash(src_o):
    return src_o-kappa**2*dslash_oe(dslash_eo(src_o))

In [13]:
# give b'_o(b__0)
b__o = (ans_o-kappa*dslash_oe(ans_e))+kappa*dslash_oe((ans_e-kappa*dslash_eo(ans_o)))

wilson dslash total time: (without malloc free memcpy) :0.000015027 sec
wilson dslash total time: (without malloc free memcpy) :0.000011609 sec
wilson dslash total time: (without malloc free memcpy) :0.000011980 sec


In [14]:
def dot(a, b):
    cp.cuda.runtime.deviceSynchronize()
    return cp.inner(a.flatten().conjugate(), b.flatten())


def diff(a, b):
    cp.cuda.runtime.deviceSynchronize()
    return cp.linalg.norm(a - b) / cp.linalg.norm(b)

In [15]:
# bistabcg
r = b__o - dslash(x_o)
r_tilde = r

wilson dslash total time: (without malloc free memcpy) :0.000016864 sec
wilson dslash total time: (without malloc free memcpy) :0.000033286 sec


In [16]:
t0 = perf_counter()
for i in range(1, int(MAX_ITER)):
    # print("## rho:", rho)
    rho = dot(r_tilde, r)
    # print("## beta:", beta)
    beta = (rho / rho_prev) * (alpha / omega)
    p = r + (p - v * omega) * beta
    # v = A * p
    v = dslash(p)
    # print("## alpha:", alpha)
    alpha = rho / dot(r_tilde, v)
    s = r - v * alpha
    # t = A * s
    t = dslash(s)
    # print("## omega:", omega)
    omega = dot(t, s) / dot(t, t)
    x_o = x_o + p * alpha + s * omega
    r = s - t * omega
    r_norm2 = cp.linalg.norm(r)
    print("##{}# r_norm2:{}".format(i, r_norm2))
    # break
    if (r_norm2 < TOL or i == MAX_ITER - 1):
        print("### turns:", i)
        break
    rho_prev = rho
t1 = perf_counter()

wilson dslash total time: (without malloc free memcpy) :0.000016385 sec
wilson dslash total time: (without malloc free memcpy) :0.000014627 sec
wilson dslash total time: (without malloc free memcpy) :0.000018988 sec
wilson dslash total time: (without malloc free memcpy) :0.000023030 sec
##1# r_norm2:52.74894185893398
wilson dslash total time: (without malloc free memcpy) :0.000016185 sec
wilson dslash total time: (without malloc free memcpy) :0.000011394 sec
wilson dslash total time: (without malloc free memcpy) :0.000015183 sec
wilson dslash total time: (without malloc free memcpy) :0.000015755 sec
##2# r_norm2:22.83692687808014
wilson dslash total time: (without malloc free memcpy) :0.000024504 sec
wilson dslash total time: (without malloc free memcpy) :0.000011988 sec
wilson dslash total time: (without malloc free memcpy) :0.000026879 sec
wilson dslash total time: (without malloc free memcpy) :0.000066603 sec
##3# r_norm2:135.57009095293103
wilson dslash total time: (without malloc 

In [17]:
print("### ans_o = ", ans_o[0, 0, 0, 0, :, :])
print("### x_o = ", x_o[0, 0, 0, 0, :, :])
print('## difference: ', diff(x_o, ans_o))
print(f'Qcu time: {t1 - t0} sec')

### ans_o =  [[0.46108017+0.75854788j 0.28530865+0.32198883j 0.41571254+0.6236552j ]
 [0.41260585+0.07384592j 0.1749582 +0.42117554j 0.77390744+0.28225653j]
 [0.09300366+0.62293015j 0.96927904+0.13452909j 0.25641439+0.05939245j]
 [0.80683136+0.01895045j 0.34001844+0.1901666j  0.66271276+0.41083579j]]
### x_o =  [[0.46108017+0.75854788j 0.28530865+0.32198883j 0.41571254+0.6236552j ]
 [0.41260585+0.07384592j 0.1749582 +0.42117554j 0.77390744+0.28225653j]
 [0.09300366+0.62293015j 0.96927904+0.13452909j 0.25641439+0.05939245j]
 [0.80683136+0.01895045j 0.34001844+0.1901666j  0.66271276+0.41083579j]]
## difference:  2.0029385103086167e-11
Qcu time: 308.2201197300019 sec


In [18]:
x_e = ans_e-kappa*dslash_eo(ans_o)+kappa * \
    dslash_eo(x_o)  # equal to x_o == ans_o
print(diff(x_e, ans_e))

wilson dslash total time: (without malloc free memcpy) :0.000015656 sec
wilson dslash total time: (without malloc free memcpy) :0.000010331 sec
1.976644916438385e-11
